# Klassifikation von Pistazien

In [30]:
"""Importieren der benötigten Bibliotheken"""
import pandas as pd
import arff
import numpy as np
import matplotlib.pyplot as plt

In [31]:
def check_requirements():
    """This function checks if all required files are present in the data folder."""
    import os

    print('Überprüfe ob alle benötigten Dateien vorhanden sind...')

    req_files = [
        "data/Pistachio_16_Features_Dataset_Citation_Request.txt",
        "data/Pistachio_16_Features_Dataset.arff",
        "data/Pistachio_16_Features_Dataset.xls",
    ]

    # Sicherstellen, dass der Ordner "data" existiert
    if not os.path.isdir('data'):
        raise AssertionError('Der Ordner "data" existiert nicht.')

    # Sicherstellen, dass alle benötigten Dateien im Ordner "data" existieren
    for file in req_files:
        if not os.path.isfile(file):
            raise AssertionError(f'Die Datei "{file}" existiert nicht.')

    print('... Alle benötigten Dateien sind vorhanden.')
    print('Viel Spaß!')
    pass

In [32]:
check_requirements()

Überprüfe ob alle benötigten Dateien vorhanden sind...
... Alle benötigten Dateien sind vorhanden.
Viel Spaß!


## Einführung in den Datensatz
Es wird ein Datensatz benutzt welcher in dem [Progress of Nutrition Vol. 23 No. 2 (2021)](https://doi.org/10.23751/pn.v23i2.9686) veröffentlicht wurde.<br>
Der Datensatz wurde in dem Paper [Classification of Pistachio Species Using Improved k-NN Classifier](https://www.mattioli1885journals.com/index.php/progressinnutrition/article/view/9686/9178) erstellt.

In dem Datensatz gibt es sowohl eine ausführung mit 16 Features als auch eine mit 28 Features. 
- <b>In diesem Projekt wird die Version mit 16 Features verwendet.</b>
    <br>(Die restlichen 12 Features sind "Farben Merkmale" und werden in diesem Projekt nicht verwendet.)

- <b>Morphological Features (12 Features):</b>
    1. Area
    2. Perimeter
    3. Major_Axis
    4. Minor_Axis
    5. Eccentricity
    6. Eqdiasq
    7. Solidity
    8. Convex_Area
    9. Extent
    10. Aspect_Ratio
    11. Roundness
    12. Compactness
- <b>Shape Features (4 Features)</b>
    13. Shapefactor_1
    14. Shapefactor_2
    15. Shapefactor_3
    16. Shapefactor_4


## Vorbereitung des Datensatzes
Um mit dem Datensatz besser arbeiten zu können, werden folgende Schritte durchlaufen:
- Es wird sich zuerst angeschaut wie der Datensatz aufgebaut ist. → [Explorative Datenanalyse](#Explorative-Datenanalyse)
- Anschließend muss der Datensatz ggf. bereinigt/aufbereitet werden. → [Aufbereitung der Daten](#Aufbereitung-der-Daten)
- Zuletzt wird der Datensatz noch in Trainings- und Testdaten aufgeteilt. → [Aufteilung der Daten in Trainings- und Testdaten](#Aufteilung-der-Daten-in-Trainings--und-Testdaten)

Danach kann es mit der Klassifikation losgehen. → [Klassifikation](#Klassifikation)

In [33]:
# Laden der Daten aus dem Excel-File in ein Pandas DataFrame
file_path = 'data/Pistachio_16_Features_Dataset.xls'
data_frame = pd.read_excel(file_path)

# Anzeigen der ersten 5 Zeilen des Datensatzes, darunter die Namen der Attribute und die ersten 4 Datensätze
print(data_frame.head(), '\n')

rows = data_frame.shape[0]
columns = data_frame.shape[1] - 1  # Klassifikationsspalte wird nicht mitgezählt

print(f"Anzahl der Attribute: {columns}")
print(f"Anzahl der Datensätze: {rows}")

print('\n', data_frame.Class.value_counts(), '\n')

print(data_frame.PERIMETER.describe())



    AREA  PERIMETER  MAJOR_AXIS  MINOR_AXIS  ECCENTRICITY   EQDIASQ  SOLIDITY  \
0  63391   1568.405    390.3396    236.7461        0.7951  284.0984    0.8665   
1  68358   1942.187    410.8594    234.7525        0.8207  295.0188    0.8765   
2  73589   1246.538    452.3630    220.5547        0.8731  306.0987    0.9172   
3  71106   1445.261    429.5291    216.0765        0.8643  300.8903    0.9589   
4  80087   1251.524    469.3783    220.9344        0.8823  319.3273    0.9657   

   CONVEX_AREA  EXTENT  ASPECT_RATIO  ROUNDNESS  COMPACTNESS  SHAPEFACTOR_1  \
0        73160  0.6394        1.6488     0.3238       0.7278         0.0062   
1        77991  0.6772        1.7502     0.2277       0.7181         0.0060   
2        80234  0.7127        2.0510     0.5951       0.6767         0.0061   
3        74153  0.7028        1.9879     0.4278       0.7005         0.0060   
4        82929  0.7459        2.1245     0.6425       0.6803         0.0059   

   SHAPEFACTOR_2  SHAPEFACTOR_3  SHAPE

### Explorative Datenanalyse
Die erste Frage, die sich stellt, lautet: <b>Wie ist der Datensatz aufgebaut?</b>

| Frage                                           | Ermittlung                               |                                               Antwort |
|:------------------------------------------------|------------------------------------------|------------------------------------------------------:|
| Wie viele Attribute gibt es?                    | `print(columns)`                         |                                                    16 |
| Wie viele Datensätze gibt es?                   | `print(rows)`                            |                                                  2148 |
| Wie viele Klassen gibt es?                      | `print(data_frame.Class.value_counts())` | <table><tr><th>Kirmizi</th><th>Siit</th></tr></table> |
| Wie viele Datensätze sind pro Klasse vorhanden? | `print(data_frame.Class.value_counts())` |     <table><tr><td>1232</td><td>916</td></tr></table> |

Des Weiteren, <b>Wie sehen die Daten pro Klasse und pro Attribut aus?</b>

| Attribut  | Beschreibung | Code                              | Datentyp |          Min |           Max |         Mean |          Std |
|:----------|:-------------|:----------------------------------|:---------|-------------:|--------------:|-------------:|-------------:|
| AREA      | Fläche       | `data_frame.AREA.describe()`      | float64  | 29808.000000 | 124008.000000 | 79950.655493 | 13121.737799 | 
| PERIMETER | Umfang       | `data_frame.PERIMETER.describe()` | float64  |   858.363000 |   2755.049100 |  1425.971751 |   375.565503 |



### Aufbereitung der Daten

### Aufteilung der Daten in Trainings- und Testdaten


## Klassifikation

### Künstliches Neuronales Netz (KNN)